<a href="https://colab.research.google.com/github/Ethan-Jeong/test_webscraping/blob/master/project_pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U finance-datareader
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')

In [2]:
import sqlite3
conn = sqlite3.connect('./db.stock')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, content TEXT, press TEXT , stock TEXT, posi_nega TEXT)")

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# https://jsikim1.tistory.com/143
from datetime import datetime , timedelta
from dateutil.relativedelta import relativedelta 

now = datetime.now()
# gap = now - relativedelta(months=1)
gap = now - timedelta(days=1)

now = str(now)[0:10]
gap = str(gap)[0:10]

dt_index = pd.date_range(start=gap, end=now)
dt_list = dt_index.strftime("%Y%m%d").tolist()

for j in dt_list:

  date_cnt_uri = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3=402&date='+j+'&page=100'
  date_cnt_target = date_cnt_uri
  date_cnt_req  = requests.get(date_cnt_target)
  date_cnt_soup = BeautifulSoup(date_cnt_req.content,'html.parser')
  date_cnt_page = int(date_cnt_soup.select('td.on > a ')[0].get_text())
  
  uri = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3=402&date='+j+'&page='

  for page in range(1,date_cnt_page+1):    

    target = uri+str(page)
    req  = requests.get(target)
    soup = BeautifulSoup(req.content,'html.parser')
    datas = soup.select('#contentarea_left > ul.realtimeNewsList')

    for content in datas:
      
      titles = content.select(' li > dl > dd.articleSubject')
      article = content.select('li > dl > dd.articleSummary')
      article_date =  content.select('li > dl > dd.articleSummary > span.wdate ')
      article_press =  content.select('li > dl > dd.articleSummary > span.press ') 

      article_sum = list()

      for i in range(0,len(titles)-1):
        article_data = list()
        
        article_len = len(article[i].get_text(" ",strip=True))
        
        data_date = article_date[i].get_text(" ",strip=True)[0:10]
        data_time = article_date[i].get_text(" ",strip=True)[11:17]
        data_press = article_press[i].get_text(" ",strip=True)
        data_title = titles[i].get_text(" ",strip=True)
        
        if article_len > 90:
          idx = article[i].get_text(" ",strip=True).index('...')
          data_content = article[i].get_text(" ",strip=True)[0:idx+3]
        else:
          data_content = article[i].get_text(" ",strip=True)
        
        c.execute("INSERT INTO article( date , time , press , title, content, stock , posi_nega ) VALUES(?,?,?,?,?,?,?)",( data_date,data_time,data_press,data_title,data_content,'stock','posi_nega'))
        conn.commit()
c.close()

In [9]:
conn = sqlite3.connect('/content/db.stock')

c_title = conn.cursor()
c_id = conn.cursor()
c_update = conn.cursor()

c_title.execute("SELECT title FROM article ")
c_id.execute("SELECT * FROM article ")

sql_title = c_title.fetchall()
sql_id = c_id.fetchall()

title_list = [list[0] for list in sql_title ]
id_list = [list[0] for list in sql_id ]

df_krx_list = df_krx['Name'].tolist()

for k in range(0,len(title_list)):
  for l in range(0,int(len(title_list[k].split()))):
    keyword = title_list[k].split()[l]
    if keyword in df_krx_list:
      sql_update = 'update article set stock = "' + keyword +'" where id = ' + str(sql_id[k][0])
      c_update.execute(sql_update)
      conn.commit()

In [25]:
c_select = conn.cursor()
final = c_select.execute("SELECT * FROM article where stock != 'stock' ")
df = pd.DataFrame(final)
df.columns= ['id', 'date' , 'time' , 'title', 'content', 'press' ,'stock' , 'posi_nega' ]
df
c.close()

,id,date,time,title,content,press,stock,posi_nega
0,6,2021-08-10,18:27,공모가도 못넘긴 크래프톤 '굴욕',삼성증권은 초대형 투자은행(IB) 중 최초로 온라인 공모주 청약시간을 오후 10시까...,파이낸셜뉴스,크래프톤,posi_nega
1,7,2021-08-10,18:26,크래프톤 호된 신고식… 공모가 고평가 논란 다시 불붙었다 [대형주 IPO 불패 깨졌다],삼성자산운용은 지난 6일을 기준으로 업계 최초로 관리자산(AUM)이 300조원을 넘...,파이낸셜뉴스,크래프톤,posi_nega
2,22,2021-08-10,16:49,동아에스티 2분기 영업익 79억원으로 흑자전환…전문약 선방,KTB자산운용이 미국 멀티패밀리(임대주택) 대출형 상품에 주로 투자하는 5500억원...,한국경제,동아에스티,posi_nega
3,27,2021-08-10,16:03,빛바랜 '게임대장주' 대관식…공모가도 못 지킨 크래프톤,[파이낸셜뉴스] 이재용 삼성전자 부회장의 가석방 결정에도 삼성 그룹주가 전반적으로 ...,파이낸셜뉴스,크래프톤,posi_nega
4,28,2021-08-10,16:02,거래량은 터졌는데…공모가 대비 '-8.8%' 꼬리표 달게 된 크래프톤,닭가슴살 판매 플랫폼 ‘랭킹닭컴’을 보유하고 있는 푸드나무가 52주 신고가 경신을 ...,한국경제,크래프톤,posi_nega
...,...,...,...,...,...,...,...,...
89,598,2021-08-11,09:12,"미래에셋운용, 초등학생 대상 여름방학 특강.. '비대면 온라인교육' 성료","[서울경제] 코로나19 발병 이후 신규 확진자가 처음으로 2,200명을 넘어서면서 ...",서울경제,대상,posi_nega
90,599,2021-08-11,09:10,"청약서 크래프톤 이긴 원티드랩, 상장 첫날 '따상'도 성공",포스코가 미국 1조 달러 규모 인프라 투자 수혜 기대감에 상승하고 있다. 11일 오...,한국경제TV,크래프톤,posi_nega
91,601,2021-08-11,09:01,위메이드 2분기 영업이익 269억원..'흑자 전환',[헤럴드경제=이세진 기자] 스테인레스강 등을 제조하는 현대차그룹 계열사 현대비앤지스...,헤럴드경제,위메이드,posi_nega
92,608,2021-08-11,08:52,"CJ CGV, 대작 개봉에 외형 성장 가시화…목표가 25%↑-유진",[파이낸셜뉴스]SK증권은 11일 아이원스에 대해 올해 2·4분기 영업이익 91억원의...,파이낸셜뉴스,CJ,posi_nega
